In [1]:
import sys
sys.path.append("python/")

import sympy as sp
import numdifftools as nd
import numpy as np
import scipy.optimize as opt

import datlib
import symbase
import Z2num
import Z2sym

# Ферромагнетик в симметричном случае

## Выражение для $Z^{(2)}$

### Интеграл

In [2]:
h, l = Z2sym.h, Z2sym.l
s1, s2 = symbase.SigmaSymbol("\\sigma_1"), symbase.SigmaSymbol("\\sigma_2")
t, u = sp.symbols("t u")

In [3]:
expr = sp.Integral(sp.exp(t**2/(2*l) + s1*t), (t, 0, h+s2*l))
expr = expr.transform(t, (t-s1*l, t)).expand()
expr = expr.transform(t, (t*sp.sqrt(2*l), t)).factor()
display(expr)

expr = expr.replace(sp.Integral, lambda *args: sp.exp(args[1][2]**2)).expand()
expr = expr*sp.exp(-(h**2+l**2)/(2*l))
expr.expand()

sqrt(2)*sqrt(\lambda)*exp(-\lambda/2)*Integral(exp(t**2), (t, sqrt(2)*\sigma_1*sqrt(\lambda)/2, sqrt(2)*(\sigma_1*\lambda + \sigma_2*\lambda + h)/(2*sqrt(\lambda))))

sqrt(2)*sqrt(\lambda)*exp(\sigma_1*h)*exp(\sigma_2*h)*exp(\sigma_1*\sigma_2*\lambda)

### Производные $\cal{F}^{(2)}$

In [4]:
expr = s1*s2*Z2sym._sigmas_expr_E_symmetrical*Z2sym._sigmas_expr_cF2_symmetrical
expr = expr.diff(l).expand()
symbase.collect_sigmas(expr, Z2sym._sigmas_terms_symmetrical, [s1, s2])

{exp(\sigma_1*h)*exp(\sigma_2*h)*exp(\sigma_1*\sigma_2*\lambda)*F2(sqrt(2)*\sigma_1*sqrt(\lambda)/2 + sqrt(2)*\sigma_2*sqrt(\lambda)/2 + sqrt(2)*h/(2*sqrt(\lambda))): {(1,
   1): (-2*\lambda**2 + h**2)/(2*\lambda**2)},
 exp(\sigma_1*h)*exp(\sigma_2*h)*exp(\sigma_1*\sigma_2*\lambda): {(0,
   1): sqrt(2)/(4*sqrt(\lambda)),
  (1, 0): sqrt(2)/(4*sqrt(\lambda)),
  (1, 1): -sqrt(2)*h/(4*\lambda**(3/2))}}

In [5]:
display(Z2sym.E_symmetrical(h, l, 1, 1).diff(h).rewrite(sp.exp))
display(Z2sym.E_symmetrical(h, l, 1, 1).rewrite(sp.exp).diff(h))

2*exp(\lambda)*exp(2*h) - 2*exp(\lambda)*exp(-2*h)

2*exp(\lambda)*exp(2*h) - 2*exp(\lambda)*exp(-2*h)

In [6]:
display(Z2sym.cF2_symmetrical(h, l).diff(l).rewrite(sp.exp).expand())
display(Z2sym.cF2_symmetrical(h, l).rewrite(sp.exp).diff(l).expand())

-exp(\lambda)*exp(2*h)*F2(sqrt(2)*sqrt(\lambda) + sqrt(2)*h/(2*sqrt(\lambda))) + exp(\lambda)*exp(-2*h)*F2(sqrt(2)*sqrt(\lambda) - sqrt(2)*h/(2*sqrt(\lambda))) + 2*exp(-\lambda)*F2(sqrt(2)*h/(2*sqrt(\lambda))) + h**2*exp(\lambda)*exp(2*h)*F2(sqrt(2)*sqrt(\lambda) + sqrt(2)*h/(2*sqrt(\lambda)))/(2*\lambda**2) - h**2*exp(\lambda)*exp(-2*h)*F2(sqrt(2)*sqrt(\lambda) - sqrt(2)*h/(2*sqrt(\lambda)))/(2*\lambda**2) - h**2*exp(-\lambda)*F2(sqrt(2)*h/(2*sqrt(\lambda)))/\lambda**2 + sqrt(2)*exp(\lambda)*exp(2*h)/(2*sqrt(\lambda)) - sqrt(2)*exp(\lambda)*exp(-2*h)/(2*sqrt(\lambda)) - sqrt(2)*h*exp(\lambda)*exp(2*h)/(4*\lambda**(3/2)) - sqrt(2)*h*exp(\lambda)*exp(-2*h)/(4*\lambda**(3/2)) + sqrt(2)*h*exp(-\lambda)/(2*\lambda**(3/2))

-exp(\lambda)*exp(2*h)*F2(sqrt(2)*sqrt(\lambda) + sqrt(2)*h/(2*sqrt(\lambda))) + exp(\lambda)*exp(-2*h)*F2(sqrt(2)*sqrt(\lambda) - sqrt(2)*h/(2*sqrt(\lambda))) + 2*exp(-\lambda)*F2(sqrt(2)*h/(2*sqrt(\lambda))) + h**2*exp(\lambda)*exp(2*h)*F2(sqrt(2)*sqrt(\lambda) + sqrt(2)*h/(2*sqrt(\lambda)))/(2*\lambda**2) - h**2*exp(\lambda)*exp(-2*h)*F2(sqrt(2)*sqrt(\lambda) - sqrt(2)*h/(2*sqrt(\lambda)))/(2*\lambda**2) - h**2*exp(-\lambda)*F2(sqrt(2)*h/(2*sqrt(\lambda)))/\lambda**2 + sqrt(2)*exp(\lambda)*exp(2*h)/(2*sqrt(\lambda)) - sqrt(2)*exp(\lambda)*exp(-2*h)/(2*sqrt(\lambda)) - sqrt(2)*h*exp(\lambda)*exp(2*h)/(4*\lambda**(3/2)) - sqrt(2)*h*exp(\lambda)*exp(-2*h)/(4*\lambda**(3/2)) + sqrt(2)*h*exp(-\lambda)/(2*\lambda**(3/2))

### Численная проверка

In [7]:
h, l = 1, 1.4
print(Z2num.calc_Z2_integrate(0, h, h, l)*np.exp(-2*h-l))
print(Z2num.calc_from_coeffs_symmetrical(h, l).Z2_norm)

11.129141015312976
11.129142009015116


In [8]:
h, l = Z2sym.h, Z2sym.l
cF2_diff_h = sp.lambdify((h, l), Z2sym.cF2_symmetrical(h, l).diff(h), Z2sym.num_module)
cF2_diff_l = sp.lambdify((h, l), Z2sym.cF2_symmetrical(h, l).diff(l), Z2sym.num_module)

args = 1, 1.8
gradient = nd.Gradient(lambda args: Z2num.cF2_symmetrical(*args))(args)
print(gradient[0])
print(cF2_diff_h(*args))
print()
print(gradient[1])
print(cF2_diff_l(*args))

18.342584883167927
18.3425941684832

8.179230830337767
8.179237578451808


In [9]:
h, l = Z2sym.h, Z2sym.l
sigmas_args = 1, 1
E_diff_h = sp.lambdify((h, l), Z2sym.E_symmetrical(h, l, *sigmas_args).diff(h), Z2sym.num_module)
E_diff_l = sp.lambdify((h, l), Z2sym.E_symmetrical(h, l, *sigmas_args).diff(l), Z2sym.num_module)

args = 1, 1.8
sigmas_args = list(map(bool, sigmas_args))

gradient = nd.Gradient(lambda args: Z2num.E_symmetrical(*args, *sigmas_args))(args)

print(gradient[0])
print(E_diff_h(*args))
print()
print(gradient[1])
print(E_diff_l(*args))

87.76490748044182
87.76490748044566

45.850513022822085
45.85051302282197


## Моменты

### Символьные вычисления

In [10]:
h, l = Z2sym.h, Z2sym.l
Z2_symmetrical = sp.sqrt(2)*(2*sp.pi)**2*Z2sym.cF2_symmetrical(h, l)/(h*sp.sqrt(l))

In [11]:
m, eta = Z2sym.m, Z2sym.eta
Z2 = Z2_symmetrical
Z2

4*sqrt(2)*pi**2*cF2_symmetrical(h, \lambda)/(sqrt(\lambda)*h)

In [12]:
expr_m = sp.log(Z2).diff(h)/2
expr_m = expr_m.expand()
display(sp.Eq(m, expr_m))
sp.print_latex(sp.Eq(m, expr_m))

Eq(\left<m\right>, -1/(2*h) - h/(2*\lambda) + sqrt(2)*E_symmetrical(h, \lambda, 1, 1)/(4*sqrt(\lambda)*cF2_symmetrical(h, \lambda)))

\left<m\right> = - \frac{1}{2 h} - \frac{h}{2 \lambda} + \frac{\sqrt{2} {\cal E}_{12}\left(h, \lambda\right)}{4 \sqrt{\lambda} {\cal F}^{(2)}\left(h,\lambda\right)}


In [13]:
expr_eta = sp.log(Z2).diff(l)
expr_eta = expr_eta.expand()
display(sp.Eq(eta, expr_eta))
sp.print_latex(sp.Eq(eta, expr_eta))

Eq(\left<\eta\right>, -1 - 1/(2*\lambda) + h**2/(2*\lambda**2) + sqrt(2)*E_symmetrical(h, \lambda, 1, 0)/(2*sqrt(\lambda)*cF2_symmetrical(h, \lambda)) - sqrt(2)*h*E_symmetrical(h, \lambda, 1, 1)/(4*\lambda**(3/2)*cF2_symmetrical(h, \lambda)))

\left<\eta\right> = -1 - \frac{1}{2 \lambda} + \frac{h^{2}}{2 \lambda^{2}} + \frac{\sqrt{2} {\cal E}_{1}\left(h, \lambda\right)}{2 \sqrt{\lambda} {\cal F}^{(2)}\left(h,\lambda\right)} - \frac{\sqrt{2} h {\cal E}_{12}\left(h, \lambda\right)}{4 \lambda^{\frac{3}{2}} {\cal F}^{(2)}\left(h,\lambda\right)}


In [14]:
_E12, _E1 = Z2sym.E_symmetrical(h, l, 1, 1), Z2sym.E_symmetrical(h, l, 1, 0)
_E12_E1 = sp.solve([m-expr_m, eta-expr_eta], _E12, _E1)
_expr_E12 = _E12_E1[_E12].factor()
_expr_E1 = _E12_E1[_E1].factor()

_expr_cF2_1 = sp.solve(_E12-_expr_E12, Z2sym.cF2_symmetrical(h, l))[0]
_expr_cF2_2 = sp.solve(_E1-_expr_E1, Z2sym.cF2_symmetrical(h, l))[0]

display(sp.Eq(Z2sym.cF2_symmetrical(h, l), _expr_cF2_1))
sp.print_latex(sp.Eq(Z2sym.cF2_symmetrical(h, l), _expr_cF2_1))
display(sp.Eq(Z2sym.cF2_symmetrical(h, l), _expr_cF2_2))
sp.print_latex(sp.Eq(Z2sym.cF2_symmetrical(h, l), _expr_cF2_2))


Eq(cF2_symmetrical(h, \lambda), sqrt(2)*sqrt(\lambda)*h*E_symmetrical(h, \lambda, 1, 1)/(2*(2*\lambda*\left<m\right>*h + \lambda + h**2)))

{\cal F}^{(2)}\left(h,\lambda\right) = \frac{\sqrt{2} \sqrt{\lambda} h {\cal E}_{12}\left(h, \lambda\right)}{2 \left(2 \lambda \left<m\right> h + \lambda + h^{2}\right)}


Eq(cF2_symmetrical(h, \lambda), sqrt(2)*sqrt(\lambda)*E_symmetrical(h, \lambda, 1, 0)/(2*(\lambda*\left<\eta\right> + \lambda + \left<m\right>*h + 1)))

{\cal F}^{(2)}\left(h,\lambda\right) = \frac{\sqrt{2} \sqrt{\lambda} {\cal E}_{1}\left(h, \lambda\right)}{2 \left(\lambda \left<\eta\right> + \lambda + \left<m\right> h + 1\right)}


In [15]:
subs_to_m_and_eta_symmetrical = {
    Z2sym.E_symmetrical(h, l, 1, 1): _expr_E12,
    Z2sym.E_symmetrical(h, l, 1, 0): _expr_E1,
    Z2sym.cF2_symmetrical(h, l): _expr_cF2_1
}
subs_to_m_and_eta = subs_to_m_and_eta_symmetrical

In [16]:
expr_eta2 = Z2.diff(l, 2)/Z2
expr_eta2 = expr_eta2.expand()

expr = expr_eta2.subs(subs_to_m_and_eta).expand()
expr = expr.collect([Z2sym.E_symmetrical(h, l, 0, 0)/Z2sym.E_symmetrical(h, l, 1, 1), m, eta], sp.factor)
display(expr)
sp.print_latex(expr)

\left<\eta\right>*(-3*\lambda + h**2)/(2*\lambda**2) + \left<m\right>*h*(\lambda + 1)/\lambda**2 + (2*\lambda**2 + 1)/(2*\lambda**2) - (2*\lambda*\left<m\right>*h + \lambda + h**2)*E_symmetrical(h, \lambda, 0, 0)/(2*\lambda**2*E_symmetrical(h, \lambda, 1, 1))

\frac{\left<\eta\right> \left(- 3 \lambda + h^{2}\right)}{2 \lambda^{2}} + \frac{\left<m\right> h \left(\lambda + 1\right)}{\lambda^{2}} + \frac{2 \lambda^{2} + 1}{2 \lambda^{2}} - \frac{\left(2 \lambda \left<m\right> h + \lambda + h^{2}\right) {\cal E}_{}\left(h, \lambda\right)}{2 \lambda^{2} {\cal E}_{12}\left(h, \lambda\right)}


In [17]:
expr_m_eta = Z2.diff(h, l)/(2*Z2)
expr_m_eta = expr_m_eta.expand()

expr = expr_m_eta.subs(subs_to_m_and_eta).expand()
expr = expr.collect([Z2sym.E_symmetrical(h, l, 0, 0)/Z2sym.E_symmetrical(h, l, 1, 1), m, eta], sp.factor)
display(expr)
sp.print_latex(expr)

-\left<\eta\right>*(\lambda + h**2)/(2*\lambda*h) - \left<m\right>/\lambda + (2*\lambda*\left<m\right>*h + \lambda + h**2)*E_symmetrical(h, \lambda, 0, 0)/(2*\lambda*h*E_symmetrical(h, \lambda, 1, 1)) - 1/(2*\lambda*h)

- \frac{\left<\eta\right> \left(\lambda + h^{2}\right)}{2 \lambda h} - \frac{\left<m\right>}{\lambda} + \frac{\left(2 \lambda \left<m\right> h + \lambda + h^{2}\right) {\cal E}_{}\left(h, \lambda\right)}{2 \lambda h {\cal E}_{12}\left(h, \lambda\right)} - \frac{1}{2 \lambda h}


In [18]:
_E = Z2sym.E_symmetrical(h, l, 0, 0)/Z2sym.E_symmetrical(h, l, 1, 1)
subs_to_m_eta_symmetrical = {
    _E: sp.solve(expr_m_eta.subs(subs_to_m_and_eta)-Z2sym.m_eta, _E)[0]
}
subs_to_m_eta = subs_to_m_eta_symmetrical

In [19]:
subs_to_upsilon = {Z2sym.m_eta: m*(1-2*Z2sym.upsilon)}

In [20]:
expr = expr_eta2.subs(subs_to_m_and_eta).subs(subs_to_m_eta).expand()
subs_to_eta2 = {l: sp.solve(expr-Z2sym.eta2, l)[0]}
expr = sp.Eq(l/2*(Z2sym.eta2-1), sp.expand(l/2*(expr.subs(subs_to_upsilon)-1)))

display(expr)
sp.print_latex(expr)

Eq(\lambda*({\left<\eta^2\right>} - 1)/2, \Upsilon*\left<m\right>*h - \left<\eta\right>)

\frac{\lambda \left({\left<\eta^2\right>} - 1\right)}{2} = \Upsilon \left<m\right> h - \left<\eta\right>


In [21]:
for moment, expr_moment in zip([m, eta], [expr_m, expr_eta]):
    for arg in [h, l]:
        expr = expr_moment.diff(arg).expand().subs(subs_to_m_and_eta).expand()
        expr = expr.subs(subs_to_m_eta).factor().expand()
        expr = expr.subs(subs_to_eta2).factor().expand()
        display(sp.Eq(sp.Derivative(moment, arg), expr))

Eq(Derivative(\left<m\right>, h), \left<\eta\right> - 2*\left<m\right>**2 - 2*\left<m\right>/h + 1)

Eq(Derivative(\left<m\right>, \lambda), -\left<\eta\right>*\left<m\right> + {\left<m\eta\right>})

Eq(Derivative(\left<\eta\right>, h), -2*\left<\eta\right>*\left<m\right> + 2*{\left<m\eta\right>})

Eq(Derivative(\left<\eta\right>, \lambda), -\left<\eta\right>**2 + {\left<\eta^2\right>})

### Численная проверка

In [22]:
def Z2_symmetrical_integrate(args):
    h, l = args
    return Z2num.calc_Z2_integrate(0, h, h, l)

In [23]:
f_expr_m = sp.lambdify((h, l), expr_m, Z2sym.num_module_norm)
f_expr_eta = sp.lambdify((h, l), expr_eta, Z2sym.num_module_norm)

args = 4, 5
moments = nd.Gradient(Z2_symmetrical_integrate)(args)/Z2_symmetrical_integrate(args)
z2 = Z2num.calc_from_coeffs_symmetrical(*args)

print(moments[0]/2)
print(f_expr_m(*args))
print(z2.m)
print()
print(moments[1])
print(f_expr_eta(*args))
print(z2.eta)

0.8371722011551646
0.8371718561570379
0.8371718561570379

0.8546061112004999
0.8546056972028968
0.8546056972028968


In [24]:
f_expr_cF2_1 = sp.lambdify((h, l, m, eta), _expr_cF2_1, Z2sym.num_module)
f_expr_cF2_2 = sp.lambdify((h, l, m, eta), _expr_cF2_2, Z2sym.num_module)

args = 1, 1.6
args_moments = f_expr_m(*args), f_expr_eta(*args)

print(Z2num.cF2_symmetrical(*args))
print(f_expr_cF2_1(*args, *args_moments))
print(f_expr_cF2_2(*args, *args_moments))

8.34716277372662
8.347162773726618
8.34716277372662


In [25]:
f_expr_m_eta = sp.lambdify((h, l), expr_m_eta, Z2sym.num_module_norm)
f_expr_eta2 = sp.lambdify((h, l), expr_eta2, Z2sym.num_module_norm)

args = 3.4, 1.9
moments = nd.Hessian(Z2_symmetrical_integrate)(args)/Z2_symmetrical_integrate(args)
z2 = Z2num.calc_from_coeffs_symmetrical(*args)

print(moments[0, 1]/2)
print(moments[1, 0]/2)
print(f_expr_m_eta(*args))
print(z2.m_eta)
print()
print(moments[1, 1])
print(f_expr_eta2(*args))
print(z2.eta2)

0.5815164055484547
0.5815164055484547
0.5815162841798034
0.5815162841798025

0.5911376785139352
0.5911371476339244
0.5911371476339227


## Предельные случаи 

### $\lambda \rightarrow 0$

In [26]:
expr = Z2.rewrite(sp.exp)
expr = Z2sym.dawsn_to_series_oo(expr, 1).factor()
expr = sp.limit(expr, l, 0).expand()
display(expr)

expr.rewrite(sp.sin).factor()

4*pi**2*exp(2*h)/h**2 - 8*pi**2/h**2 + 4*pi**2*exp(-2*h)/h**2

8*pi**2*(cosh(2*h) - 1)/h**2

In [27]:
expr = expr_m.rewrite(sp.exp)
expr = Z2sym.dawsn_to_series_oo(expr, 2).factor()
expr = sp.limit(expr, l, 0).factor()
display(expr)

subs_to_m_llb = {
    sp.exp(2*h): sp.solve(expr.expand()-m, sp.exp(2*h))[0]
}

expr.expand().rewrite(sp.tanh).factor().expand()

(h*exp(2*h) + h - exp(2*h) + 1)/(h*(exp(h) - 1)*(exp(h) + 1))

1/tanh(h) - 1/h

In [28]:
expr = expr_eta.rewrite(sp.exp)
expr = Z2sym.dawsn_to_series_oo(expr, 3).factor()
expr = sp.limit(expr, l, 0).factor()
display(expr)

expr.expand().subs(subs_to_m_llb).factor()

(h*exp(2*h) + h - exp(2*h) + 1)**2/(h**2*(exp(h) - 1)**2*(exp(h) + 1)**2)

\left<m\right>**2

In [29]:
expr = expr_m_eta.rewrite(sp.exp)
expr = Z2sym.dawsn_to_series_oo(expr, 4).factor()
expr = sp.limit(expr, l, 0).factor()
display(expr)

expr = expr.expand().subs(subs_to_m_llb).factor().expand()
display(expr)
sp.print_latex(expr)

(h*exp(2*h) + h - exp(2*h) + 1)*(h**2*exp(2*h) - h**2 - 2*h*exp(2*h) - 2*h + 2*exp(2*h) - 2)/(h**3*(exp(h) - 1)**2*(exp(h) + 1)**2)

-2*\left<m\right>**2/h + \left<m\right>

- \frac{2 \left<m\right>^{2}}{h} + \left<m\right>


In [30]:
expr = expr_eta2.rewrite(sp.exp).factor()
expr = Z2sym.dawsn_to_series_oo(expr, 5).factor()
expr = sp.limit(expr, l, 0).factor()
display(expr)

expr = expr.expand().subs(subs_to_m_llb).factor().expand()
display(expr)
sp.print_latex(expr)

(h**4*exp(4*h) - 2*h**4*exp(2*h) + h**4 - 4*h**3*exp(4*h) + 4*h**3 + 10*h**2*exp(4*h) + 4*h**2*exp(2*h) + 10*h**2 - 12*h*exp(4*h) + 12*h + 6*exp(4*h) - 12*exp(2*h) + 6)/(h**4*(exp(h) - 1)**2*(exp(h) + 1)**2)

6*\left<m\right>**2/h**2 - 4*\left<m\right>/h + 1

\frac{6 \left<m\right>^{2}}{h^{2}} - \frac{4 \left<m\right>}{h} + 1


In [31]:
# expr = expr_m.diff(l).rewrite(sp.exp)
# expr = Z2sym.dawsn_to_series_oo(expr, 4).factor()
# expr = sp.limit(expr, l, 0).factor()
# display(expr)

# expr.expand().subs(subs_to_m_llb).factor().expand()

In [32]:
# expr = (expr_eta).diff(l).rewrite(sp.exp)
# expr = Z2sym.dawsn_to_series_oo(expr, 5).factor()
# expr = sp.limit(expr, l, 0).factor()
# display(expr)

# expr = expr.expand().subs(subs_to_m_llb).factor().expand()
# expr

In [33]:
# expr = (expr_m_eta/2).diff(l).factor().rewrite(sp.exp)
# expr = Z2sym.dawsn_to_series_oo(expr, 6).factor()
# expr = sp.limit(expr, l, 0).factor()
# display(expr)

# expr = expr.expand().subs(subs_to_m_llb).factor().expand()
# display(expr)

# expr = expr/m - (m-2*m**2/h)/m**2*(m - m * m * m - 2 * m * m/h)
# expr = expr.expand()
# display(expr)
# display(expr.replace(m, h/3))

In [34]:
# expr = expr_eta2.diff(l).factor().rewrite(sp.exp)
# expr = Z2sym.dawsn_to_series_oo(expr, 7).factor()
# expr = sp.limit(expr, l, 0).factor()
# display(expr)

# expr = expr.expand().subs(subs_to_m_llb).factor().expand()
# display(expr)
# display(expr.replace(m, h/3))

### $h \rightarrow 0$

In [35]:
expr = Z2.rewrite(sp.exp)
expr = symbase.LHopital(expr.factor(), h).replace(h, 0).expand()
display(expr)

expr = expr.rewrite(sp.sin).factor()
display(expr)
sp.print_latex(expr)

8*pi**2*exp(\lambda)/\lambda - 8*pi**2*exp(-\lambda)/\lambda

16*pi**2*sinh(\lambda)/\lambda

\frac{16 \pi^{2} \sinh{\left(\lambda \right)}}{\lambda}


In [36]:
expr = expr_m.rewrite(sp.exp).factor()
expr = symbase.LHopital(expr, h, 2)
expr = expr.replace(h, 0)
display(expr)

0

In [37]:
expr = expr_eta.rewrite(sp.exp)
expr = symbase.LHopital(expr.factor(), h)
expr = expr.replace(h, 0).factor()
display(expr)

subs_to_eta_prmg = {
    sp.exp(2*l): sp.solve(expr.expand()-eta, sp.exp(2*l))[0]
}

expr = expr.expand().rewrite(sp.tanh).factor().expand()
display(expr)
sp.print_latex(expr)

(\lambda*exp(2*\lambda) + \lambda - exp(2*\lambda) + 1)/(\lambda*(exp(\lambda) - 1)*(exp(\lambda) + 1))

1/tanh(\lambda) - 1/\lambda

\frac{1}{\tanh{\left(\lambda \right)}} - \frac{1}{\lambda}


In [38]:
expr = ((1-expr_m_eta/expr_m)/2)
expr = symbase.LHopital(expr.factor(), h, 2)
expr = expr.rewrite(sp.exp).replace(h, 0).factor()
display(expr)

expr.expand().subs(subs_to_eta_prmg).factor()

(\lambda*exp(2*\lambda) + \lambda - exp(2*\lambda) + 1)/(\lambda*(2*\lambda*exp(2*\lambda) - exp(2*\lambda) + 1))

\left<\eta\right>/(\lambda*(\left<\eta\right> + 1))

In [39]:
expr = expr_eta2.rewrite(sp.exp)
expr = symbase.LHopital(expr.factor(), h, 2)
expr = expr.rewrite(sp.exp).replace(h, 0).factor()
display(expr)

expr.expand().subs(subs_to_eta_prmg).factor().expand()

(\lambda**2*exp(2*\lambda) - \lambda**2 - 2*\lambda*exp(2*\lambda) - 2*\lambda + 2*exp(2*\lambda) - 2)/(\lambda**2*(exp(\lambda) - 1)*(exp(\lambda) + 1))

1 - 2*\left<\eta\right>/\lambda

In [40]:
expr = expr_m/h
expr = symbase.LHopital(expr.factor(), h, 2)
expr = expr.rewrite(sp.exp).replace(h, 0).factor()
display(expr)

expr.expand().subs(subs_to_eta_prmg).factor()

(2*\lambda*exp(2*\lambda) - exp(2*\lambda) + 1)/(3*\lambda*(exp(\lambda) - 1)*(exp(\lambda) + 1))

(\left<\eta\right> + 1)/3

In [41]:
# expr = (expr_m/h).diff(h).factor()
# expr = symbase.LHopital(expr, h, 5)
# expr = expr.rewrite(sp.exp).replace(h, 0)
# display(expr)

In [42]:
expr = expr_eta.diff(h).factor()
expr = symbase.LHopital(expr, h, 2)
expr = expr.rewrite(sp.exp).replace(h, 0)
display(expr)

0

In [43]:
# expr = ((1-expr_m_eta/expr_m)/2).diff(h).factor()
# expr = symbase.LHopital(expr.factor(), h, 6)
# expr = expr.rewrite(sp.exp).replace(h, 0).factor()
# display(expr)

## Генерация кода C++ для реализации метода Ньютона

In [44]:
subs = {
    Z2sym.cF2_symmetrical(h, l): sp.Symbol("cF2_norm"),
    Z2sym.E_symmetrical(h, l, 1, 1): sp.Symbol("E12_norm"),
    Z2sym.E_symmetrical(h, l, 1, 0): sp.Symbol("E1_norm"),
    Z2sym.E_symmetrical(h, l, 0, 0): sp.Symbol("E_norm"),
    m: sp.Symbol("m"), eta: sp.Symbol("eta"),
    sp.exp(-2*h): sp.Symbol("_exp_2h"),
    sp.exp(-2*l): sp.Symbol("_exp_2l"),
    l: sp.Symbol("l")
}


def optimize(expr):
    expr = symbase.optimize_pow(expr, {l: "l"})
    expr = expr.subs(subs)
    return expr

In [45]:
print(symbase.get_ccode(optimize(.5*(1-expr_m_eta/m)).factor()))

0.5*(-0.125*M_SQRT2*E12_norm*(h*h*h)*l + 0.125*M_SQRT2*E12_norm*h*(l*l) + 0.25*M_SQRT2*E1_norm*(h*h)*(l*l) + 0.25*M_SQRT2*E1_norm*(l*l*l) - 0.25*M_SQRT2*E_norm*h*(l*l*l) + 0.25*cF2_norm*(h*h*h*h)*sqrt_l - 0.5*cF2_norm*(h*h)*(l*l)*sqrt_l - 0.5*cF2_norm*(h*h)*l*sqrt_l + 1.0*cF2_norm*h*(l*l*l)*m*sqrt_l - 0.5*cF2_norm*(l*l*l)*sqrt_l - 0.25*cF2_norm*(l*l)*sqrt_l)/(cF2_norm*h*(l*l*l)*m*sqrt_l)


## Решение обратной задачи

### Прямая задача

In [46]:
h = np.logspace(-5, 1, 100)
l = np.logspace(-5, 1, 100)

h, l = np.meshgrid(h, l)

In [47]:
m, eta = np.empty_like(h), np.empty_like(h)
upsilon, eta2 = np.empty_like(h), np.empty_like(h)
Z2_norm = np.empty_like(m)

for i in range(h.size):
    z2 = Z2num.calc_from_coeffs_symmetrical(h.flat[i], l.flat[i])
    m.flat[i], eta.flat[i] = z2.m, z2.eta
    upsilon.flat[i], eta2.flat[i] = z2.upsilon, z2.eta2
    Z2_norm.flat[i] = z2.Z2_norm

In [48]:
datlib.np2dat(
    "data/Z2.dat",
    "h l upsilon eta2 Z2_norm".split(),
    np.asarray([
        h, l, upsilon, eta2, Z2_norm
    ])
)

In [49]:
# %%bash
# gplt3 -pm3d -U 'Z2_norm' data/Z2.dat -ln xy -to data/picts/pict.pdf 

### Обратная задача

In [50]:
m = np.linspace(0, 1, 100)
zeta = np.linspace(0, 1, 100)

m, zeta = np.meshgrid(m, zeta)
eta = zeta + (1-zeta)*m**2

In [51]:
h, l = np.empty_like(m), np.empty_like(m)
upsilon, eta2, mh2 = np.empty_like(m), np.empty_like(m), np.empty_like(m)
psi0, Z2_norm = np.empty_like(m), np.empty_like(m)

for i in range(m.size):
    z2 = Z2num.find_coeffs_symmetrical(m.flat[i], eta.flat[i])
    h.flat[i], l.flat[i] = z2.h, z2.l
    upsilon.flat[i], eta2.flat[i], mh2.flat[i] = z2.upsilon, z2.eta2, z2.mh2
    psi0.flat[i], Z2_norm.flat[i] = z2.psi0, z2.Z2_norm

In [52]:
datlib.np2dat(
    "data/Z2.dat",
    "m eta zeta h l upsilon eta2 mh2 psi0 Z2_norm".split(),
    np.asarray([
        m, eta, zeta, h, l, upsilon, eta2, mh2, psi0, Z2_norm
    ])
)

In [53]:
# %%bash
# gplt3 -pm3d -U 'Z2_norm(m,zeta)' data/Z2.dat -to data/picts/pict.pdf 

### Асимптотика при $\left<m\right> \rightarrow 1$

In [54]:
m = np.linspace(.99, 1, 100)
zeta = np.linspace(.05, 1, 100)

m, zeta = np.meshgrid(m, zeta)
eta = zeta + (1-zeta)*m**2

In [55]:
h, l = np.empty_like(m), np.empty_like(m)
upsilon, eta2, mh2 = np.empty_like(m), np.empty_like(m), np.empty_like(m)
psi0, Z2_norm = np.empty_like(m), np.empty_like(m)

for i in range(m.size):
    z2 = Z2num.find_coeffs_symmetrical_by_scipy(m.flat[i], eta.flat[i])
    h.flat[i], l.flat[i] = z2.h, z2.l
    upsilon.flat[i], eta2.flat[i], mh2.flat[i] = z2.upsilon, z2.eta2, z2.mh2
    psi0.flat[i], Z2_norm.flat[i] = z2.psi0, z2.Z2_norm

/home/daiwery/projects/Z-analytic/python/Z2num.py:26: RuntimeWarning: divide by zero encountered in scalar divide
  return .5/(1-m), .5/(1-eta)


In [56]:
datlib.np2dat(
    "data/Z2.dat",
    "m eta zeta h l upsilon eta2 mh2 psi0 Z2_norm".split(),
    np.asarray([
        m, eta, zeta, h, l, upsilon, eta2, mh2, psi0, Z2_norm
    ])
)

In [57]:
# %%bash
# gplt3 -pm3d -U 'upsilon(m,eta)' data/Z2.dat -to data/picts/pict.pdf 

In [58]:
variables = 1-m, 1-eta
y_data = upsilon
indexes_nans = np.logical_not(np.isnan(h))

def func(_, *p_args, nans=True):    
    result = p_args[0]
    result += p_args[1]*variables[0] + p_args[2]*variables[1]

    if nans:
        return result[indexes_nans].ravel()
    else:
        return result
    

p0 = [0] + 2*[1]
p = opt.curve_fit(func, None, y_data[indexes_nans].ravel(), p0=p0)[0]
p

array([1.80037290e-06, 2.39256366e-04, 4.98311465e-01])

In [59]:
# x = 1 - m
# y = 1 - eta

# upsilon = y/2
# eta2 = 1 - 2*y
# m_par_2 = 1 - 2*x
# psi0 = -y/2

# Ферромагнетик в несимметричном случае

## Выражение для $Z^{(2)}$

### Аргументы

In [ ]:
h1, h2, l = sp.symbols("h_1 h_2 \\lambda", positive=True)
s1, s2 = symbase.SigmaSymbol("\\sigma_1"), symbase.SigmaSymbol("\\sigma_2")
t = sp.Symbol("t")

In [ ]:
arg_exp = t**2/(2*l)*h1/h2 + s1*t
bound = h2+s2*l

arg_exp = arg_exp.replace(t, t-s1*l*h2/h1).expand()
bound += s1*l*h2/h1

arg_exp = arg_exp.replace(t, t*sp.sqrt(2*l*h2/h1))
bound /= sp.sqrt(2*l*h2/h1)

display(arg_exp)
display(bound)

-\lambda*h_2/(2*h_1) + t**2

sqrt(2)*sqrt(h_1)*(\sigma_1*\lambda*h_2/h_1 + \sigma_2*\lambda + h_2)/(2*sqrt(\lambda)*sqrt(h_2))

In [ ]:
expr = bound**2 + (arg_exp - t**2)
expr = expr.expand()
expr -= (h2**2+l**2)/(2*l*h2/h1)
expr = expr.expand()

display(expr)

\sigma_1*\sigma_2*\lambda + \sigma_1*h_2 + \sigma_2*h_1

### Производные $\cal{F}^{(2)}$

In [ ]:
_sub_expr_E = sp.exp(h1*s1+h2*s2+l*s1*s2).expand()
_sub_expr_cF2 = Z2sym.F2((h2+s1*l+s2*l*h2/h1)/sp.sqrt(2*l*h2/h1)).expand()

_dummy_subs = {
    _sub_expr_E*_sub_expr_cF2: sp.Dummy(),
    _sub_expr_E: sp.Dummy()
}


def sigmas_sum(expr):
    expr = expr.expand().subs(_dummy_subs)

    terms = sp.collect(expr, _dummy_subs.values(), evaluate=False)
    for dummy in _dummy_subs.values():
        if dummy in terms:
            terms[dummy] = sp.Poly(terms[dummy], [s1, s2]).as_dict()
    for key, dummy in _dummy_subs.items():
        if dummy in terms:
            terms[key] = terms.pop(dummy)

    return terms

In [ ]:
class Z2nsSigmasFunctionBase(sp.Function):
    def __init__(self, name, *args, **kwargs):
        self.name = name

    def _latex(self, printer, exp=None):
        args = list(map(printer.doprint, self.args[:3]))
        args = f"\\left({args[0]}, {args[1]}, {args[2]}\\right)"

        indexes = ""
        for i, item in enumerate(self.args[3:]):
            if item:
                indexes += str(i+1)
        name = f"{self.name}"+"_{" + indexes + "}"

        if exp is None:
            return name+args
        else:
            exp = printer.doprint(exp)
            return f"{name}^{exp}"+args

    def get_sigmas_factor(self):
        factor = 1
        if self.args[3]:
            factor *= s1
        if self.args[4]:
            factor *= s2
        return factor

In [ ]:
class EFunction_ns(Z2nsSigmasFunctionBase):
    def __init__(self, *args, **kwargs):
        super().__init__("{\\cal E}", *args, **kwargs)

    def fdiff(self, argindex=1):
        factor = self.get_sigmas_factor()
        expr = (factor*_sub_expr_E).diff([h1, h2, l][argindex-1])
        terms = sigmas_sum(expr)

        subs = dict(zip([h1, h2, l], self.args[:3]))

        result = 0
        for key, item in terms[_sub_expr_E].items():
            result += item.subs(subs)*EFunction_ns(*self.args[:3], *key)

        return result
        
    def _eval_rewrite(self, rule, args, **hints):
        h1, h2, l, arg_s1, arg_s2 = args
        if rule == sp.exp:
            result = 0
            for s1 in SIGMAS:
                for s2 in SIGMAS:
                    term = sp.exp(s1*h1+s2*h2+s1*s2*l)
                    if arg_s1:
                        term *= s1
                    if arg_s2:
                        term *= s2
                    result += term
            return result
    
    @classmethod
    def eval(cls, *args):
        if args[0] == args[1]:
            return EFunction(args[0], *args[2:])

In [ ]:
EFunction_ns(h1, h2, l, 1, 1).diff(h1)

EFunction_ns(h_1, h_2, \lambda, 0, 1)

In [ ]:
class cF2Function_ns(sp.Function):
    def _latex(self, printer, exp=None):
        args = list(map(printer.doprint, self.args[:3]))
        args = f"\\left({args[0]}, {args[1]}, {args[2]}\\right)"
        if exp is None:
            return "{\\cal F}^{(2)}"+args
        else:
            exp = printer.doprint(exp)
            return f"\\left({self._latex(printer)}\\right)^{exp}"

    def fdiff(self, argindex=1):
        expr = (s1*s2*_sub_expr_E*_sub_expr_cF2).diff([h1, h2, l][argindex-1])
        terms = sigmas_sum(expr)

        subs = dict(zip([h1, h2, l], self.args[:3]))

        result = 0
        for key, item in terms[_sub_expr_E].items():
            result += item.subs(subs)*EFunction_ns(*self.args, *key)
        
        _term = terms[_sub_expr_E*_sub_expr_cF2][(1, 1)].factor().subs(subs)
        result += _term*cF2Function_ns(*self.args)
        return result

    def _eval_rewrite(self, rule, args, **hints):
        h1, h2, l = args
        if rule == sp.exp:
            result = 0
            for s1 in SIGMAS:
                for s2 in SIGMAS:
                    term = sp.exp(s1*h1+s2*h2+s1*s2*l)
                    term *= F2Function((h2+s1*l+s2*l*h2/h1)/sp.sqrt(2*l*h2/h1))
                    term *= s1*s2
                    result += term
            return result

        return result

    @classmethod
    def eval(cls, *args):
        if args[0] == args[1]:
            return cF2Function(args[0], args[2])

In [ ]:
expr = cF2Function_ns(h1, h2, l).diff(l).expand()

display(expr)
sp.print_latex(expr)

-h_1*cF2Function_ns(h_1, h_2, \lambda)/(2*h_2) - h_2*cF2Function_ns(h_1, h_2, \lambda)/(2*h_1) + h_1*h_2*cF2Function_ns(h_1, h_2, \lambda)/(2*\lambda**2) + sqrt(2)*sqrt(h_1)*EFunction_ns(h_1, h_2, \lambda, 0, 1)/(4*sqrt(\lambda)*sqrt(h_2)) + sqrt(2)*sqrt(h_2)*EFunction_ns(h_1, h_2, \lambda, 1, 0)/(4*sqrt(\lambda)*sqrt(h_1)) - sqrt(2)*sqrt(h_1)*sqrt(h_2)*EFunction_ns(h_1, h_2, \lambda, 1, 1)/(4*\lambda**(3/2))

- \frac{h_{1} {\cal F}^{(2)}\left(h_{1}, h_{2}, \lambda\right)}{2 h_{2}} - \frac{h_{2} {\cal F}^{(2)}\left(h_{1}, h_{2}, \lambda\right)}{2 h_{1}} + \frac{h_{1} h_{2} {\cal F}^{(2)}\left(h_{1}, h_{2}, \lambda\right)}{2 \lambda^{2}} + \frac{\sqrt{2} \sqrt{h_{1}} {\cal E}_{2}\left(h_{1}, h_{2}, \lambda\right)}{4 \sqrt{\lambda} \sqrt{h_{2}}} + \frac{\sqrt{2} \sqrt{h_{2}} {\cal E}_{1}\left(h_{1}, h_{2}, \lambda\right)}{4 \sqrt{\lambda} \sqrt{h_{1}}} - \frac{\sqrt{2} \sqrt{h_{1}} \sqrt{h_{2}} {\cal E}_{12}\left(h_{1}, h_{2}, \lambda\right)}{4 \lambda^{\frac{3}{2}}}


### Численный расчет

#### Определения

In [ ]:
def _Z2_ns_E_terms(h1, h2, l):
    def func(s1, s2): return np.exp(s1*h1+s2*h2+s1*s2*l)
    return np.asarray([[func(s1, s2) for s2 in SIGMAS] for s1 in SIGMAS])

def _Z2_ns_F_terms(h1, h2, l):
    def func(s1, s2): return f_F2Function((h2+s1*l+s2*l*h2/h1)/np.sqrt(2*l*h2/h1))
    return np.asarray([[func(s1, s2) for s2 in SIGMAS] for s1 in SIGMAS])

In [ ]:
def f_cF2Function_ns(h1, h2, l):
    _terms = _Z2_ns_E_terms(h1, h2, l)*_Z2_ns_F_terms(h1, h2, l)
    def _sigmas_func_12(s1, s2): return s1*s2
    return _Z2_sum_terms(_terms, _sigmas_func_12)

def v_cF2Function_ns(x): return f_cF2Function_ns(*x)

In [ ]:
def f_Z2_ns(h1, h2, l):
    return np.sqrt(2)*(2*np.pi)**2/(np.sqrt(h1*h2*l))*f_cF2Function_ns(h1, h2, l)

In [ ]:
def raw_Z2_ns(h1, h2, l):
    def _sqrt(x): return np.sqrt(h2**2+l**2+2*h2*l*x)
    def func(x): return np.exp(h1*x)*f_Z1(_sqrt(x))
    return 2*np.pi*quad(func, -1, 1)[0]

def v_raw_Z2_ns(x): return raw_Z2_ns(*x)

In [ ]:
# def f_EFunction_ns(h1, h2, l, arg_s1, arg_s2):
#     def _sigmas_func(s1, s2):
#         result = 1
#         if arg_s1:
#             result *= s1
#         if arg_s2:
#             result *= s2
#         return result
#     return _Z2_sum_terms(_Z2_ns_E_terms(h1, h2, l), _sigmas_func)

# module_ns_dictionary = {"cF2Function_ns": f_cF2Function_ns, "EFunction_ns": f_EFunction_ns, "F2Function": f_F2Function}

#### Проверка

In [ ]:
args = 3, 2, 1.5
print(raw_Z2_ns(*args))
print(f_Z2_ns(*args))

NameError: name 'quad' is not defined

In [ ]:
f_cF2_ns_diff_h1 = sp.lambdify((h1, h2, l), cF2Function_ns(h1, h2, l).diff(h1), [module_ns_dictionary, "scipy"])
f_cF2_ns_diff_h2 = sp.lambdify((h1, h2, l), cF2Function_ns(h1, h2, l).diff(h2), [module_ns_dictionary, "scipy"])
f_cF2_ns_diff_l  = sp.lambdify((h1, h2, l), cF2Function_ns(h1, h2, l).diff(l),  [module_ns_dictionary, "scipy"])

args = 1, 2, 1.4

print(nd.directionaldiff(v_cF2Function_ns, args, [1, 0, 0], n=1))
print(f_cF2_ns_diff_h1(*args))
print()
print(nd.directionaldiff(v_cF2Function_ns, args, [0, 1, 0], n=1))
print(f_cF2_ns_diff_h2(*args))
print()
print(nd.directionaldiff(v_cF2Function_ns, args, [0, 0, 1], n=1))
print(f_cF2_ns_diff_l(*args))

16.26477756622653
16.26477756622638

14.133140858579903
14.133140858580846

14.618960735571534
14.618960735572816


## Моменты

In [ ]:
Z2_ns = sp.sqrt(2)*(2*sp.pi)**2/sp.sqrt(h1*h2*l)*cF2Function_ns(h1, h2, l)
Z2_ns

4*sqrt(2)*pi**2*cF2Function_ns(h_1, h_2, \lambda)/(sqrt(\lambda)*sqrt(h_1)*sqrt(h_2))

### $\left<m^2_h\right>$ в симметричном случае

In [ ]:
expr_m_par_2 = Z2_ns.diff(h1, 2)/Z2_ns
expr_m_par_2 = expr_m_par_2.expand()
expr_m_par_2 = expr_m_par_2.subs({h1: h, h2: h}).subs(subs_to_moments).expand()
expr_m_par_2 = expr_m_par_2.subs(subs_to_m_eta).factor().expand()
# print(expr_m_par_2.collect([EFunction(h, l, 0, 0)/EFunction(h, l, 1, 1), m, eta], sp.factor))
expr_m_par_2

In [ ]:
f_expr_m_par_2 = sp.lambdify((h, l, m, m_eta), expr_m_par_2, [module_dictionary_norm, "scipy"])

args = 10, 1.2
args_moment = f_expr_m(*args), f_expr_eta(*args)
m_eta_value = f_expr_m_eta(*args, *args_moment)

args_ns = args[0], args[0], args[1]
print(nd.directionaldiff(v_raw_Z2_ns, args_ns, [1, 0, 0], n=2)/v_raw_Z2_ns(args_ns))
print(f_expr_m_par_2(*args, args_moment[0], m_eta_value))

0.8345854948296006
0.8345854948275206


In [ ]:
expr = Z2_ns.diff(h1, h2)/Z2_ns
expr = expr.expand()
# expr = expr.subs({h1: h, h2: h}).subs(subs_to_moments).expand()
# expr = expr.subs(subs_to_m_eta).factor().expand()
expr

sqrt(2)*\lambda**(3/2)*sqrt(h_1)*EFunction_ns(h_1, h_2, \lambda, 0, 1)/(8*h_2**(5/2)*cF2Function_ns(h_1, h_2, \lambda)) - sqrt(2)*\lambda**(3/2)*EFunction_ns(h_1, h_2, \lambda, 1, 0)/(8*sqrt(h_1)*h_2**(3/2)*cF2Function_ns(h_1, h_2, \lambda)) - sqrt(2)*\lambda**(3/2)*EFunction_ns(h_1, h_2, \lambda, 0, 1)/(8*h_1**(3/2)*sqrt(h_2)*cF2Function_ns(h_1, h_2, \lambda)) + sqrt(2)*\lambda**(3/2)*sqrt(h_2)*EFunction_ns(h_1, h_2, \lambda, 1, 0)/(8*h_1**(5/2)*cF2Function_ns(h_1, h_2, \lambda)) - sqrt(2)*sqrt(\lambda)*sqrt(h_1)*EFunction_ns(h_1, h_2, \lambda, 1, 1)/(8*h_2**(3/2)*cF2Function_ns(h_1, h_2, \lambda)) + sqrt(2)*sqrt(\lambda)*EFunction_ns(h_1, h_2, \lambda, 0, 0)/(4*sqrt(h_1)*sqrt(h_2)*cF2Function_ns(h_1, h_2, \lambda)) - sqrt(2)*sqrt(\lambda)*EFunction_ns(h_1, h_2, \lambda, 0, 1)/(8*sqrt(h_1)*h_2**(3/2)*cF2Function_ns(h_1, h_2, \lambda)) - sqrt(2)*sqrt(\lambda)*sqrt(h_2)*EFunction_ns(h_1, h_2, \lambda, 1, 1)/(8*h_1**(3/2)*cF2Function_ns(h_1, h_2, \lambda)) - sqrt(2)*sqrt(\lambda)*EFuncti

### $\Psi$ в симметричном случае

#### Численный расчет

In [ ]:
from scipy.special import iv

def _raw_Z2_general(h1_per_2, h1_par_2, h2, l):
    def _sqrt(x): return np.sqrt(h2**2+l**2+2*h2*l*np.cos(x))
    def func(x): return iv(0, h1_per_2*np.sin(x))*np.exp(h1_par_2*np.cos(x))*f_Z1(_sqrt(x))*np.sin(x)
    return 2*np.pi*quad(func, 0, np.pi)[0]

def raw_Z2_general(h1, h2, l):
    h2_norm = np.linalg.norm(h2)
    h1_par_2 = h1.dot(h2)/h2_norm
    h1_per_2 = np.sqrt(np.abs(h1.dot(h1) - h1_par_2**2))
    return _raw_Z2_general(h1_per_2, h1_par_2, h2_norm, l)

In [ ]:
arg_h1 = np.asarray([0, 0, 2])
arg_h2 = np.asarray([0, 0, 1])
arg_l = 2

print(raw_Z2_general(arg_h1, arg_h2, arg_l))
print(raw_Z2_ns(2, 1, 2))

782.8469849077993
782.8469849077994


In [ ]:
def calc_diff(func, args):
    if len(args) == 0:
        return func
    func = calc_diff(func, args[1:])
    def diff(x0):
        x1 = np.copy(x0)
        x2 = np.copy(x0)
        x1[args[0]] += 1e-4
        x2[args[0]] -= 1e-4
        return (func(x1)-func(x2))/(2*1e-4)
    return diff

In [ ]:
print(calc_diff(lambda x: np.sin(x), [0, 0])([1.]))
print(np.sin(1))

[-0.84147098]
0.8414709848078965


In [ ]:
def raw_psi(h, l, nk):
    def func(args):
        h, l, gamma1, gamma2  = args
        h = np.asarray([0, 0, h])
        h1 = h + gamma1*nk
        h2 = h + gamma2*nk
        return raw_Z2_general(h1, h2, l)
    return (calc_diff(func, [1, 3, 3])([h, l, 0, 0]) - calc_diff(func, [2, 3])([h, l, 0, 0]))/raw_Z2(h, l)
    # return (calc_diff(func, [3, 3])([h, l, 0, 0]))/raw_Z2(h, l)

In [ ]:
args = 1.4, 1.6
vnk = np.asarray([0, 0, 1])

print(raw_psi(*args, vnk))
print((1-f_expr_eta(*args))*f_expr_m(*args)**3*(0.46134-1.3836))

-0.06225928895713271
-0.06146556308531417


#### Символьные вычисления

In [ ]:
from sympy.vector import CoordSys3D, Del

N = CoordSys3D("N")
delop = Del()

vh1, vh2, vh3 = sp.symbols("h_1 h_2 h_3")
vh = vh1*N.i + vh2*N.j + vh3*N.k

nk1, nk2, nk3 = sp.symbols("n_{1K} n_{2K} n_{3K}")
nk = nk1*N.i + nk2*N.j + nk3*N.k

In [ ]:
gamma1, gamma2 = sp.symbols("\\gamma_1 \\gamma_2", positive=True)

In [ ]:
v1 = vh + gamma1*nk
v2 = vh + gamma2*nk

v1_par_2 = v1.dot(v2)/v2.magnitude()
v1_per_2 = sp.sqrt(v1.dot(v1) - v1_par_2**2)
v2_magnitude = v2.magnitude()

In [ ]:
def _diff_func(func, args): 
    if len(args) == 0:
        return func

    arg = args[0]
    if isinstance(func, sp.Function):
        func = func.fdiff(arg)
        return _diff_func(func, args[1:])
    if isinstance(func, sp.Subs):
        return sp.Subs(_diff_func(func.args[0], args), *func.args[1:])
    if isinstance(func, sp.Derivative):
        return sp.Derivative(_diff_func(func.args[0], args), *func.args[1:])

In [ ]:
Z = sp.Function("Z")(v1_per_2, v1_par_2, v2_magnitude, l)

def two_diff_func(func, arg1, arg2):
    args = func.args
    result = 0
    for i, arg in enumerate(args):
        result += arg.diff(arg1, arg2).subs({gamma2: 0}).factor()*_diff_func(func, [i+1])

        factor = arg.diff(arg1).subs({gamma2: 0})
        for j, j_arg in enumerate(args):
            term = (factor*j_arg.diff(arg2)).subs({gamma2: 0}).factor()
            result += term*_diff_func(func, [i+1, j+1])
    return result

expr_psi = (two_diff_func(Z, gamma1, gamma1).diff(l) - two_diff_func(Z, gamma1, gamma2)).subs({gamma1: 0, gamma2: 0})
expr_psi = expr_psi.subs(vh.dot(vh), h**2).expand()

def v_per_diff_eval(*args):
    func = args[0]
    diff_args = args[1:]
    per_arg = func.args[0]
    non_per_diff_args = tuple(filter(lambda item: item[0] != per_arg, diff_args))
    per_diff_args = tuple(filter(lambda item: item[0] == per_arg, diff_args))

    if len(per_diff_args) == 0:
        return sp.Derivative(*args)
    
    _, n = per_diff_args[0]

    if n == 2:
        func = func - _diff_func(func, (2, 2))
        func /= 2
    elif n == 1:
        return 0
    else:
        return sp.Derivative(*args)

    if len(non_per_diff_args) > 0:
        func = sp.Derivative(func, *non_per_diff_args)

    return func

expr_psi = expr_psi.replace(sp.Derivative, v_per_diff_eval).doit()

expr_psi = expr_psi.replace(sp.Function("Z"), lambda *args: Z2_ns.subs(dict(zip([h1, h2, l], args[1:])))).doit()
expr_psi = (expr_psi/Z2_ns.subs({h1: h, h2: h})).expand()
expr_psi = expr_psi.subs(subs_to_moments)
expr_psi = expr_psi.factor()
expr_psi

-(2*h_1**2*n_{1K}**2 - h_1**2*n_{2K}**2 - h_1**2*n_{3K}**2 + 6*h_1*h_2*n_{1K}*n_{2K} + 6*h_1*h_3*n_{1K}*n_{3K} - h_2**2*n_{1K}**2 + 2*h_2**2*n_{2K}**2 - h_2**2*n_{3K}**2 + 6*h_2*h_3*n_{2K}*n_{3K} - h_3**2*n_{1K}**2 - h_3**2*n_{2K}**2 + 2*h_3**2*n_{3K}**2)*(-3*\lambda**2*\left<\eta\right>*EFunction(h, \lambda, 1, 1) + 6*\lambda**2*\left<m\right>*h*EFunction(h, \lambda, 0, 0) - 6*\lambda**2*\left<m\right>*h*EFunction(h, \lambda, 1, 1) + 3*\lambda**2*EFunction(h, \lambda, 0, 0) + 2*\lambda*\left<m\right>*h**3*EFunction(h, \lambda, 0, 0) - 2*\lambda*\left<m\right>*h**3*EFunction(h, \lambda, 1, 1) - 6*\lambda*\left<m\right>*h*EFunction(h, \lambda, 1, 1) + 4*\lambda*h**2*EFunction(h, \lambda, 0, 0) - 3*\lambda*EFunction(h, \lambda, 1, 1) - \left<\eta\right>*h**4*EFunction(h, \lambda, 1, 1) - 2*\left<m\right>*h**3*EFunction(h, \lambda, 1, 1) + h**4*EFunction(h, \lambda, 0, 0) - h**2*EFunction(h, \lambda, 1, 1))/(8*\lambda**2*h**4*EFunction(h, \lambda, 1, 1))

In [ ]:
f_expr_psi = sp.lambdify((vh1, vh2, vh3, nk1, nk2, nk3, h, l, m, eta), expr_psi, module_dictionary)
 
args = 1.4, 1.6
vh_args =  np.asarray([0, 0, args[0]])
nk_args = np.asarray([0, 1, 1.])
nk_args /= np.linalg.norm(nk_args)
args_moment = f_expr_m(*args), f_expr_eta(*args)

print(f_expr_psi(*vh_args, *nk_args, *args, *args_moment))
print(raw_psi(*args, nk_args))
print((1-f_expr_eta(*args))*f_expr_m(*args)**3*(0.46134-1.3836*(vh_args/np.linalg.norm(vh_args)).dot(nk_args)**2))

-0.015562017813131329
-0.01560789008725465
-0.015359392870385248


In [ ]:
expr_psi_upsilon = expr_psi.expand().subs(subs_to_m_eta).factor().expand()
expr_psi_upsilon.factor()

(3*\lambda*\left<m\right> - 3*\lambda*{\left<m\eta\right>} - 2*\left<\eta\right>*h + \left<m\right>*h**2 - h**2*{\left<m\eta\right>})*(2*h_1**2*n_{1K}**2 - h_1**2*n_{2K}**2 - h_1**2*n_{3K}**2 + 6*h_1*h_2*n_{1K}*n_{2K} + 6*h_1*h_3*n_{1K}*n_{3K} - h_2**2*n_{1K}**2 + 2*h_2**2*n_{2K}**2 - h_2**2*n_{3K}**2 + 6*h_2*h_3*n_{2K}*n_{3K} - h_3**2*n_{1K}**2 - h_3**2*n_{2K}**2 + 2*h_3**2*n_{3K}**2)/(4*\lambda*h**3)

In [ ]:
expr_psi_upsilon_Q0 = expr_psi_upsilon.replace(l, 2/(eta2-1)*((m-m_eta)/2*h-eta)).factor()
expr_psi_upsilon_Q0

(3*\left<m\right> + h*{\left<\eta^2\right>} - h - 3*{\left<m\eta\right>})*(2*h_1**2*n_{1K}**2 - h_1**2*n_{2K}**2 - h_1**2*n_{3K}**2 + 6*h_1*h_2*n_{1K}*n_{2K} + 6*h_1*h_3*n_{1K}*n_{3K} - h_2**2*n_{1K}**2 + 2*h_2**2*n_{2K}**2 - h_2**2*n_{3K}**2 + 6*h_2*h_3*n_{2K}*n_{3K} - h_3**2*n_{1K}**2 - h_3**2*n_{2K}**2 + 2*h_3**2*n_{3K}**2)/(4*h**3)

In [ ]:
expr_psi_nk = expr_psi_upsilon_Q0.args[-1]
expr_psi_nk

2*h_1**2*n_{1K}**2 - h_1**2*n_{2K}**2 - h_1**2*n_{3K}**2 + 6*h_1*h_2*n_{1K}*n_{2K} + 6*h_1*h_3*n_{1K}*n_{3K} - h_2**2*n_{1K}**2 + 2*h_2**2*n_{2K}**2 - h_2**2*n_{3K}**2 + 6*h_2*h_3*n_{2K}*n_{3K} - h_3**2*n_{1K}**2 - h_3**2*n_{2K}**2 + 2*h_3**2*n_{3K}**2

In [ ]:
expr = expr_psi_nk-(3*(vh.dot(nk)**2)-vh.dot(vh))
expr = expr.expand()
expr = expr.subs({nk1**2: 1-nk2**2-nk3**2})
expr.expand()

0

In [ ]:
expr = expr_psi_upsilon_Q0/expr_psi_nk*h**2
expr = expr.factor().expand()

display(expr)
sp.print_latex(expr)

3*\left<m\right>/(4*h) + {\left<\eta^2\right>}/4 - 1/4 - 3*{\left<m\eta\right>}/(4*h)

\frac{3 \left<m\right>}{4 h} + \frac{{\left<\eta^2\right>}}{4} - \frac{1}{4} - \frac{3 {\left<m\eta\right>}}{4 h}
